<a href="https://colab.research.google.com/github/KILjungjoon/Project_Asian_Parliament/blob/main/%E7%AB%8B%E6%B3%95%E5%A7%94%E5%93%A1%EC%83%81%EC%84%B8%EC%A0%95%EB%B3%B4parsing(%E7%AC%AC10%E5%B1%86).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 입법원 홈페이지에서 입법의원 정보 추출하기
* 중요한 코드 설명</p>
```
委員會_list = []
for li in left_infos[6].find_all('li'):
    委員會_list.append(li.text.strip())
委員會.append(','.join(委員會_list))
到職日期.append(left_infos[-1].text.strip().split('：')[1])
```
* 委員會는 left_infos[6]이지만 그 안에는 li 요소가 불규칙적으로 있다. 따라서 到職日期는 left_infos[-1]을 지정해줘야 한다.





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Selenium 설치
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launch

In [13]:
!pip install beautifulsoup4
!pip install tqdm

In [14]:
# Selenium 모듈 불러와서 driver 설정하기
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [15]:
# 필요한 모듈 불러오기
import os
import glob
import time
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [17]:
# 第10屆 立法委員 메인 페이지 정보 parsing
url="https://www.ly.gov.tw/Pages/List.aspx?nodeid=109"
driver.get(url)
time.sleep(1)

html=driver.page_source
soup=BeautifulSoup(html, 'html.parser')

# 의원 정보가 담긴 구간 추출
contents = soup.find_all('div', class_='content row')

In [18]:
# 현직의원과 이직의원 그룹 나누기
divs_incumbent = contents[0].find_all('div', class_='col-xs-4 col-sm-3 col-md-3 col-lg-15')
divs_resigned = contents[1].find_all('div', class_='col-xs-4 col-sm-3 col-md-3 col-lg-15')

In [ ]:
# 실제 정보를 추출하는 코드
姓名=[]
英文姓名=[]
屆別=[]
性別=[]
黨籍=[]
選區=[]
委員會=[]
到職日期=[]
學歷=[]
經歷=[]
電話=[]
傳真=[]
通訊處=[]
備註=[]
區分=[]
URL=[]

# 현직의원 정보 처리 : divs_incumbent
for div in tqdm(divs_incumbent, desc='현직의원 정보 추출 :'):
    # 의원 개별 url 추출
    member_url = "https://www.ly.gov.tw" + div.find('a')['href']
    # driver로 url 읽고 soup 생성
    driver.get(member_url)
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # 이름 추출
    legislatorname = soup.find('div', class_='legislatorname').text.strip()
    姓名.append(legislatorname)

    # 좌측 정보 구간 추출
    info_left = soup.find('div', class_='info-left')
    left_infos = info_left.find_all('li')

    英文姓名.append(left_infos[0].text.strip().split('：')[1])
    屆別.append(left_infos[1].text.strip().split('：')[1])
    性別.append(left_infos[2].text.strip().split('：')[1])
    黨籍.append(left_infos[3].text.strip().split('：')[1])
    選區.append(left_infos[5].text.strip().split('：')[1])
    委員會_list = []
    for li in left_infos[6].find_all('li'):
        委員會_list.append(li.text.strip())
    委員會.append(','.join(委員會_list))
    到職日期.append(left_infos[-1].text.strip().split('：')[1])

    # 우측 정보 구간 추출
    info_right = soup.find('div', class_='col-sm-8 info-right')
    right_titles = []
    right_contents = []
    for h4 in info_right.find_all('h4', class_='title'):
        right_titles.append(h4.text.strip())
    for ul in info_right.find_all('ul'):
        right_contents.append(ul.find_all('li'))

    # 學歷~通訊處 추출
    for i in range(len(right_titles)):
        if right_titles[i] == '學歷':
            學歷.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '經歷':
            經歷.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '電話':
            電話.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '傳真':
            傳真.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '通訊處':
            通訊處.append(','.join([li.text.strip() for li in right_contents[i]]))
    備註.append('')
    區分.append('現任')
    URL.append(member_url)

# 이직의원 정보 처리 : divs_resigned
for div in tqdm(divs_resigned, desc='이직의원 정보 추출 :'):
    # 의원 개별 url 추출
    member_url = "https://www.ly.gov.tw" + div.find('a')['href']
    # driver로 url 읽고 soup 생성
    driver.get(member_url)
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # # 이름 추출
    # legislatorname = soup.find('div', class_='legislatorname').text.strip()
    # 姓名.append(legislatorname)

    # 좌측 정보 구간 추출
    info_left = soup.find('div', class_='info-left')
    left_infos = info_left.find_all('li')

    英文姓名.append(left_infos[0].text.strip().split('：')[1])
    屆別.append(left_infos[1].text.strip().split('：')[1])
    性別.append(left_infos[2].text.strip().split('：')[1])
    黨籍.append(left_infos[3].text.strip().split('：')[1])
    選區.append(left_infos[5].text.strip().split('：')[1])
    委員會_list = []
    for li in left_infos[6].find_all('li'):
        委員會_list.append(li.text.strip())
    委員會.append(','.join(委員會_list))
    到職日期.append(left_infos[-1].text.strip().split('：')[1])

    # 우측 정보 구간 추출
    info_right = soup.find('div', class_='col-sm-8 info-right')
    right_titles = []
    right_contents = []
    for h4 in info_right.find_all('h4', class_='title'):
        right_titles.append(h4.text.strip())
    for ul in info_right.find_all('ul'):
        right_contents.append(ul.find_all('li'))

    # 學歷~通訊處 추출
    for i in range(len(right_titles)):
        if right_titles[i] == '學歷':
            學歷.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '經歷':
            經歷.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '電話':
            電話.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '傳真':
            傳真.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '通訊處':
            通訊處.append(','.join([li.text.strip() for li in right_contents[i]]))
        elif right_titles[i] == '備註':
            備註.append(','.join([li.text.strip() for li in right_contents[i]]))
    區分.append('離職')
    URL.append(member_url)

In [ ]:
import pandas as pd
df_new = pd.DataFrame({
    '姓名': 姓名,
    '英文姓名': 英文姓名,
    '屆別': 屆別,
    '性別': 性別,
    '黨籍': 黨籍,
    '選區': 選區,
    '委員會': 委員會,
    '到職日期': 到職日期,
    '學歷': 學歷,
    '經歷': 經歷,
    '電話': 電話,
    '傳真': 傳真,
    '通訊處': 通訊處,
    '備註': 備註,
    '區分': 區分,
    'URL': URL
})
df_new
df_new.to_excel('.../Taiwan/legislators_10th.xlsx', index=False)